# Anime Prediction with Cosine similarity

In [1]:
#load required libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Upload nad explore the data set

In [2]:
#Upload the dataset prepared
df=pd.read_csv("Anime_dataset_all.csv")
df

,Title,English,Synonyms,Japanese,Type,Episodes,Status,Aired,Premiered,Broadcast,...,Source,Genres,Duration,Rating,Score,Ranked,Popularity,Members,Favorites,link
0,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist: Brotherhood,Hagane no Renkinjutsushi: Fullmetal Alchemist...,鋼の錬金術師 FULLMETAL ALCHEMIST,TV,64,Finished Airing,"Apr 5, 2009 to Jul 4, 2010",Spring 2009,Sundays at 17:00 (JST),...,Manga,"Action, Military, Adventure, Comedy, Drama, Ma...",24 min. per ep.,R - 17+ (violence & profanity),9.161,#122,#3,"25,72,173","1,82,659",https://myanimelist.net/anime/5114/Fullmetal_A...
1,Gintama: The Final,NaN,NaN,銀魂 THE FINAL,Movie,1,Finished Airing,"Jan 8, 2021",NaN,NaN,...,Manga,"Action, Sci-Fi, Comedy, Historical, Parody, Dr...",1 hr. 44 min.,PG-13 - Teens 13 or older,9.101,#222,#2256,"59,478","1,687",https://myanimelist.net/anime/39486/Gintama__T...
2,Shingeki no Kyojin Season 3 Part 2,Attack on Titan Season 3 Part 2,NaN,進撃の巨人 Season3 Part.2,TV,10,Finished Airing,"Apr 29, 2019 to Jul 1, 2019",Spring 2019,Mondays at 00:10 (JST),...,Manga,"Action, Drama, Fantasy, Military, Mystery, Sho...",23 min. per ep.,R - 17+ (violence & profanity),9.101,#322,#36,"14,82,102","42,292",https://myanimelist.net/anime/38524/Shingeki_n...
3,Steins;Gate,Steins;Gate,NaN,STEINS;GATE,TV,24,Finished Airing,"Apr 6, 2011 to Sep 14, 2011",Spring 2011,Wednesdays at 02:05 (JST),...,Visual novel,"Sci-Fi, Psychological, Drama, Thriller",24 min. per ep.,PG-13 - Teens 13 or older,9.101,#422,#11,"20,16,145","1,55,022",https://myanimelist.net/anime/9253/Steins_Gate
4,Fruits Basket: The Final,NaN,"Fruits Basket 3rd Season, Fruits Basket (2019...",フルーツバスケット The Final,TV,13,Finished Airing,"Apr 6, 2021 to Jun 29, 2021",Spring 2021,Tuesdays at 01:30 (JST),...,Manga,"Slice of Life, Comedy, Supernatural, Drama, Ro...",23 min. per ep.,PG-13 - Teens 13 or older,9.091,#522,#712,"2,36,860","10,476",https://myanimelist.net/anime/42938/Fruits_Bas...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18344,Jashin Shoukan: Inran Kyonyuu Oyako Ikenie Gis...,NaN,NaN,邪娠娼館―淫乱巨乳母娘生贄儀式―,OVA,2,Not yet aired,"Sep 3, 2021",NaN,NaN,...,Visual novel,Hentai,20 min. per ep.,Rx - Hentai,NaN,N/A22,#11563,780,3,https://myanimelist.net/anime/49020/Jashin_Sho...
18345,Konbini Shoujo Z,NaN,NaN,コンビニ少女Z,OVA,Unknown,Not yet aired,Not available,NaN,NaN,...,Visual novel,Hentai,30 min.,Rx - Hentai,NaN,N/A22,#12863,482,1,https://myanimelist.net/anime/42383/Konbini_Sh...
18346,Korogashi Ryouta,NaN,NaN,ころがし涼太,OVA,3,Finished Airing,"Nov 21, 1990 to Nov 21, 1991",NaN,NaN,...,Unknown,"Action, Comedy, Hentai, Seinen",41 min. per ep.,R+ - Mild Nudity,NaN,N/A22,#14489,282,1,https://myanimelist.net/anime/10564/Korogashi_...
18347,Kuroinu II The Animation,NaN,NaN,黒獣[クロイヌ]II THE ANIMATION,OVA,Unknown,Not yet aired,"Aug 27, 2021 to ?",NaN,NaN,...,Visual novel,"Hentai, Demons, Fantasy",30 min.,Rx - Hentai,NaN,N/A22,#8564,"2,631",6,https://myanimelist.net/anime/49095/Kuroinu_II...


In [3]:
df.shape

(18349, 23)

In [4]:
df.columns

Index(['Title', 'English', 'Synonyms', 'Japanese', 'Type', 'Episodes',
       'Status', 'Aired', 'Premiered', 'Broadcast', 'Producers', 'Licensors',
       'Studios', 'Source', 'Genres', 'Duration', 'Rating', 'Score', 'Ranked',
       'Popularity', 'Members', 'Favorites', 'link'],
      dtype='object')

In [5]:
df.drop(['Aired','Premiered','Broadcast','Duration','Favorites','link'],axis = 1,inplace=True)

In [6]:
df.isnull().sum()

Title             0
English       11023
Synonyms       7351
Japanese         65
Type              0
Episodes          0
Status            0
Producers         0
Licensors         0
Studios           0
Source            0
Genres            0
Rating            0
Score          5239
Ranked            0
Popularity        0
Members           0
dtype: int64

In [7]:
#Dealing with empty cells
features = ['Licensors', 'Genres', 'Studios', 'Rating','Score']

for feature in features:
    df[feature] = df[feature].fillna('')

In [8]:
df.dtypes

Title         object
English       object
Synonyms      object
Japanese      object
Type          object
Episodes      object
Status        object
Producers     object
Licensors     object
Studios       object
Source        object
Genres        object
Rating        object
Score         object
Ranked        object
Popularity    object
Members       object
dtype: object

In [9]:
#converting the datatype so combine all the required columns to prepare cosine simarity matrix
df.Rating = df.Rating.astype(str)
df.Score=df.Score.astype(str)

In [58]:
def combined_features(row):
    return row['Licensors']+" "+row['Genres']+" "+row['Studios']+" "+row['Rating']+" "+row['Score']+" "+row['Type']
df["combined_features"] = df.apply(combined_features, axis =1)

In [59]:
#Combined all required columns 
df.head()

,Title,English,Synonyms,Japanese,Type,Episodes,Status,Producers,Licensors,Studios,Source,Genres,Rating,Score,Ranked,Popularity,Members,combined_features
Index,,,,,,,,,,,,,,,,,,
0,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist: Brotherhood,Hagane no Renkinjutsushi: Fullmetal Alchemist...,鋼の錬金術師 FULLMETAL ALCHEMIST,TV,64,Finished Airing,"Aniplex, Square Enix, Mainichi Broadcasting Sy...","Funimation, Aniplex of America",Bones,Manga,"Action, Military, Adventure, Comedy, Drama, Ma...",R - 17+ (violence & profanity),9.161,#122,#3,"25,72,173","Funimation, Aniplex of America Action, Milita..."
1,Gintama: The Final,NaN,NaN,銀魂 THE FINAL,Movie,1,Finished Airing,"TV Tokyo, Warner Bros. Japan",Eleven Arts,Bandai Namco Pictures,Manga,"Action, Sci-Fi, Comedy, Historical, Parody, Dr...",PG-13 - Teens 13 or older,9.101,#222,#2256,"59,478","Eleven Arts Action, Sci-Fi, Comedy, Historica..."
2,Shingeki no Kyojin Season 3 Part 2,Attack on Titan Season 3 Part 2,NaN,進撃の巨人 Season3 Part.2,TV,10,Finished Airing,"Production I.G, Dentsu, Mainichi Broadcasting ...",Funimation,Wit Studio,Manga,"Action, Drama, Fantasy, Military, Mystery, Sho...",R - 17+ (violence & profanity),9.101,#322,#36,"14,82,102","Funimation Action, Drama, Fantasy, Military, ..."
3,Steins;Gate,Steins;Gate,NaN,STEINS;GATE,TV,24,Finished Airing,"Frontier Works, Media Factory, Movic, AT-X, Ka...",Funimation,White Fox,Visual novel,"Sci-Fi, Psychological, Drama, Thriller",PG-13 - Teens 13 or older,9.101,#422,#11,"20,16,145","Funimation Sci-Fi, Psychological, Drama, Thri..."
4,Fruits Basket: The Final,NaN,"Fruits Basket 3rd Season, Fruits Basket (2019...",フルーツバスケット The Final,TV,13,Finished Airing,"TV Tokyo, TV Osaka, Nihon Ad Systems, Hakusens...",Funimation,TMS Entertainment,Manga,"Slice of Life, Comedy, Supernatural, Drama, Ro...",PG-13 - Teens 13 or older,9.091000000000001,#522,#712,"2,36,860","Funimation Slice of Life, Comedy, Supernatura..."


## Creating recommendation model with cosine similarity 

Using sklearn.feature_extraction module converting the "Combined features" to the format supported by machine learning algorithms. Using CountVectorizer’s fit.tranform to count the number of texts and print the transformed matrix count_matrix into an array for better understanding.

In [60]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df["combined_features"])
print("Count Matrix:", count_matrix.toarray())

Count Matrix: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


With Cosine Similarity from Sklearn calculating the similarity between two anime. Cosine similarity is a metric used to measure how similar two items are. The output value ranges from 0–1 where 0 means no similarity and 1 means that both the items are 100% similar. 

In [61]:
cosine_sim = cosine_similarity(count_matrix)

#### Taking user input anime name and recommending 10 titles with cosine similarity

In [62]:
def get_index_from_title(a): #Getting indx of title that user has given
    return df.index[df.Title == x].values[0]

def get_title_from_index(index): # Getting the recommended titles with index
    return df[df.index == index]["Title"].values[0]


In [63]:
def print_anime_recommendation(Anime_user_likes):
    Anime_index = get_index_from_title(Anime_user_likes)
    similar_anime = list(enumerate(cosine_sim[Anime_index]))
    sorted_similar_anime = sorted(similar_anime, key=lambda x:x[1], reverse=True)
    i=0
    for anime in sorted_similar_anime:
        print(get_title_from_index(anime[0]))
        i=i+1
        if i>10:
            break

In [67]:
x=input()
print_anime_recommendation(x)

 One Piece


One Piece
One Piece Movie 14: Stampede
One Piece: Episode of East Blue - Luffy to 4-nin no Nakama no Daibouken
One Piece: Episode of Sabo - 3 Kyoudai no Kizuna Kiseki no Saikai to Uketsugareru Ishi
One Piece: Episode of Sorajima
Dragon Ball Z
Dragon Ball Kai
Dragon Ball Kai (2014)
Dragon Ball Super
One Piece: Adventure of Nebulandia
Dragon Ball Movie 1: Shen Long no Densetsu


## Conclusion: 

#### Given input title as "One Piece." Above are the titles suggested as recommendation based on the input.